#### Описание задачи

- Подробное описание в файле gpn-cup-2021-data_science_task.docx

Для увеличения продаж товаров из следующих групп:
- вода
- сладкие газированные напитки, холодный чай
- кофейные напитки с молоком
- энергетические напитки
- снеки
- соки и сокосодержащие напитки  

Вам необходимо разработать рекомендательную систему, которая будет предлагать покупателям 20 дополнительных товаров в чек.  

Пример: покупатель приходит на кассу с 2-мя товарами: напитком «Local-Cola» и чипсами «Sya'l». Алгоритм должен предложить 20 товаров, которые пользователь вероятнее всего захочет добавить в свою корзину (в порядке убывания релевантности). На практике кассир предложит 1й по порядку товар, из имеющихся в наличии. 
В качестве метрики качества рекомендаций используется mean average precision at 20

Транзакционные данные продаж – transactions:
- sku_id – уникальный идентификатор товара
- price – цена, по которой был продан товар
- number – количество товаров (если не топливо) 
- cheque_id – уникальный идентификатор чека
- litrs – количество литров (если товар - топливо)
- client_id – уникальный идентификатор клиента (если клиент «представился» при покупке)
- shop_id – уникальный идентификатор магазина
- date – дата транзакции  

Данные о товарах – nomenclature:
- sku_id – уникальный идентификатор товара
- full_name – полное наименование товара
- brand – наименование торговой марки
- sku_group – группа, к которой принадлежит товар
- OTM – признак собственной торговой марки
- units – единица измерения для количества 
- country – страна производства товара  

Формат всех источников - .parquet.

In [5]:
#№! pip install pyarrow

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import dates as mdates
import seaborn as sns
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [3]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


#### Загрузка данных

In [8]:
df_trans = pd.read_parquet("Data/transactions.parquet")

In [9]:
df_trans.head()

,sku_id,price,number,cheque_id,litrs,client_id,shop_id,date
0,1158,0.002335,0.000000,3338297,0.147929,78634.0,102,2171-07-23
1,1158,0.002317,0.000000,3386107,0.134562,20900.0,101,2171-07-23
2,1913,0.007850,0.000452,1845331,0.104183,96397.0,36,2171-07-23
3,1808,0.008979,0.000452,2256499,0.104183,103560.0,89,2171-07-23
4,1158,0.002355,0.000000,3257281,0.115023,67691.0,58,2171-07-23


In [10]:
df_nomencl = pd.read_parquet("Data/nomenclature.parquet")

In [11]:
df_nomencl.head()

,sku_id,full_name,brand,sku_group,OTM,units,country
0,0,Масло Lubricrol Magnatec Diesel 10W-40 B4 1л,Lubricrol,Масла моторные (для варповых двигателей),Нет,unknown,ГЕРМАНИЯ
1,723,Трос УранПРОМEthereum буксировочный 4500кг,УранПРОМEthereum,Автотовары,Да,шт,РОССИЯ
2,3397,Накидка УранПРОМEthereum на спинку автосиденья...,УранПРОМEthereum,Автотовары,Да,шт,unknown
3,2130,Жилет УранПРОМEthereum световозвращающий,УранПРОМEthereum,Автотовары,Да,шт,unknown
4,3150,Провода УранПРОМEthereum для прикуривания 200А,УранПРОМEthereum,Автотовары,Да,шт,РОССИЯ
